PyGaze Analysis

In [1]:
# We load the original results from the file data/4/processed-data/fixationsAll.csv

import pandas as pd

# We load the original results from the file data/4/processed-data/fixationsAll.csv
fixationsAll = pd.read_csv('results/original_data_4.csv')

# We only keep the columns Subject, Sex, Age, Image, CategoryName, list_size, Likert_list_size, working_size, Likert_working_size, total_size, Likert_total_size and StimulusType
fixationsAll = fixationsAll[['Subject', 'Sex', 'Age', 'Image', 'CategoryName', 'list_size', 'Likert_list_size', 'working_size', 'Likert_working_size', 'total_size', 'Likert_total_size', 
'StimulusType']]
fixationsAll['Subject'] = fixationsAll['Subject'].astype("string")
fixationsAll['Image'] = fixationsAll['Image'].astype('int32')

# We load the PyGazeAnalyzer results from the file results/pygaze_fixations_4.csv
pygaze_fixations = pd.read_csv('results/pygaze_fixations_4.csv')
# We add a 'S' before each subject number
pygaze_fixations['Participant'] = 'S' + pygaze_fixations['Participant'].astype(str)
# We remove the '.BMP' from the Task column and cast them to int32
pygaze_fixations['Task'] = pygaze_fixations['Task'].str.replace('.BMP', '')
pygaze_fixations['Task'] = pygaze_fixations['Task'].astype("int32")

# We combine the two dataframes into one by combining the columns where the values of Participant and Task are the same
results_pygaze = pd.merge(fixationsAll, pygaze_fixations, how='inner', left_on=['Subject', 'Image'], right_on=['Participant', 'Task'])
results_pygaze


C:\Users\Timon\AppData\Local\Temp\ipykernel_20236\2936749557.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pygaze_fixations['Task'] = pygaze_fixations['Task'].str.replace('.BMP', '')


,Subject,Sex,Age,Image,CategoryName,list_size,Likert_list_size,working_size,Likert_working_size,total_size,Likert_total_size,StimulusType,Participant,Task,Fixation Count,Total Fixation Duration [ms],Average Fixation Duration [ms]
0,S151,M,21.0,8,List,6.5,1,16.0,4.0,12.0,1.0,ainsert,S151,8,33,2436.1,73.821212
1,S151,M,21.0,25,List,13.0,3,6.0,1.0,13.0,1.0,ainsert,S151,25,38,4037.3,106.244737
2,S151,M,21.0,28,List,14.0,3,8.0,2.0,14.0,2.0,ainsert,S151,28,69,6723.4,97.440580
3,S151,M,21.0,21,List,14.0,3,10.0,2.0,14.0,2.0,ainsert,S151,21,25,3620.5,144.820000
4,S151,M,21.0,11,List,17.0,4,13.0,3.0,17.0,3.0,ainsert,S151,11,36,4988.3,138.563889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,S204,M,24.0,43,Tree,9.0,2,9.0,3.0,1.0,2.0,traversal,S204,43,48,4237.7,88.285417
2591,S204,M,24.0,49,Tree,9.0,2,9.0,3.0,1.0,2.0,traversal,S204,49,39,4738.2,121.492308
2592,S204,M,24.0,40,Tree,12.0,2,12.0,3.0,12.0,2.0,traversal,S204,40,78,7107.6,91.123077
2593,S204,M,24.0,55,Tree,13.0,2,13.0,3.0,13.0,2.0,traversal,S204,55,46,5105.2,110.982609


In [5]:
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu

def show_results1(results):
	# We calculate the mean and the standard deviation of the fixation count for each CategoryName
		# We first calculate the mean for each participant

	results_mean_per_subject = results.groupby(['CategoryName', 'Subject']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]
	results_std_per_subject = results.groupby(['CategoryName', 'Subject']).std(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]

	#print(results_mean_per_subject)
	#print(results_mean_per_subject.loc["List"].shape, results_mean_per_subject.loc["Mental"].shape, results_mean_per_subject.loc["Tree"].shape)


	results_mean = results_mean_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]
	results_std = results_std_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]

	#print(results_mean)

	total_fixation_times = results.groupby(['CategoryName', 'Subject'])['Total Fixation Duration [ms]'].sum()


	#results_mean = results.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]
	#results_std = results.groupby(['CategoryName']).std(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]

	# Apply Pairwise Comparisons Using Non-Parametric Wilcox Tests (α = 0.05)
	# TODO: check if there is a way to use wilcox 



	stat_list_vs_mental_fix_count, p_list_vs_mental_fix_count = mannwhitneyu(results_mean_per_subject.loc["List"]['Fixation Count'], results_mean_per_subject.loc["Mental"]['Fixation Count'])

	# Use mannwhitneyu for the other comparison
	stat_tree_vs_mental_fix_count, p_tree_vs_mental_fix_count = wilcoxon(results_mean_per_subject.loc["Tree"]['Fixation Count'], results_mean_per_subject.loc["Mental"]['Fixation Count'])

	stat_list_vs_mental_fix_time, p_list_vs_mental_fix_time = mannwhitneyu(total_fixation_times.loc["List"], total_fixation_times.loc["Mental"])

	# Use mannwhitneyu for the other comparison
	stat_tree_vs_mental_fix_time, p_tree_vs_mental_fix_time = wilcoxon(total_fixation_times.loc["Tree"], total_fixation_times.loc["Mental"])

	# We create a table to show the results

	# We divide the fixation time by 1000 to get the fixation time in seconds
	results_mean['Total Fixation Duration [ms]'] = results_mean['Total Fixation Duration [ms]']/1000
	results_std['Total Fixation Duration [ms]'] = results_std['Total Fixation Duration [ms]']/1000

	data = [

		[
			'Fixation Time (s)', 
			f"{'%.2f' % (total_fixation_times.loc['List']/1000).mean()}({'%.2f' % (total_fixation_times.loc['List']/1000).std()})",
			f"{'%.2f' % (total_fixation_times.loc['Mental']/1000).mean()}({'%.2f' % (total_fixation_times.loc['Mental']/1000).std()})",
			f"{'%.2f' % (total_fixation_times.loc['Tree']/1000).mean()}({'%.2f' % (total_fixation_times.loc['Tree']/1000).std()})",
			'%.3f' % p_list_vs_mental_fix_time,
			'%.3f' % p_tree_vs_mental_fix_time
		],
		[
			'Fixation Count', 
			f"{'%.0f' % results_mean.loc['List']['Fixation Count']} ({'%.0f' % results_std.loc['List']['Fixation Count']})", 
			f"{'%.0f' % results_mean.loc['Mental']['Fixation Count']} ({'%.0f' % results_std.loc['Mental']['Fixation Count']})", 
			f"{'%.0f' % results_mean.loc['Tree']['Fixation Count']} ({'%.0f' % results_std.loc['Tree']['Fixation Count']})", 
			'%.3f' % p_list_vs_mental_fix_count, 
			'%.3f' % p_tree_vs_mental_fix_count
		]

		
	]
	data_df = pd.DataFrame(data)
	data_df.columns = ['', 'List', 'Mental', 'Tree', 'List vs Mental p', 'Tree vs Mental p']
	return data_df

show_results1(results_pygaze)

c:\Users\Timon\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:3337: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,,List,Mental,Tree,List vs Mental p,Tree vs Mental p
0,Fixation Time (s),181.45(96.70),71.70(47.94),170.85(109.38),0.000,0.000
1,Fixation Count,49 (27),21 (14),47 (24),0.000,0.000


In [26]:

from pandas import Interval

def show_results2(results):

	results_mean_per_subject = results.groupby(['CategoryName', 'Subject']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]
	results_std_per_subject = results.groupby(['CategoryName', 'Subject']).std(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]

	#print(results_mean_per_subject)
	#print(results_mean_per_subject.loc["List"].shape, results_mean_per_subject.loc["Mental"].shape, results_mean_per_subject.loc["Tree"].shape)


	results_mean = results_mean_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]
	results_std = results_std_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]

	# Group the results by the different values of Sex
	results_sex = results.groupby("Sex")[["Fixation Count", "Average Fixation Duration [ms]"]]
	results_sex_mean = results_sex.mean(numeric_only=True)
	results_sex_std = results_sex.std(numeric_only=True)

	total_fixation_times = results.groupby(['CategoryName', 'Subject'])['Total Fixation Duration [ms]'].sum()

	# Group the results by ages between 18-22 and 23-27
	results_age_group_mean = results.groupby(['CategoryName', 'Subject']).mean(numeric_only=True)[["Fixation Count", "Average Fixation Duration [ms]", "Age"]]
	results_age = results_age_group_mean.groupby(pd.cut(results_age_group_mean["Age"], [18, 22, 27]))[["Fixation Count", "Average Fixation Duration [ms]"]]
	results_age_mean = results_age.mean(numeric_only=True)
	print(results_age_group_mean[results_age_group_mean['Age'] <= 22])
	results_age_std = results_age.std(numeric_only=True)

	# Perform non-parametric Wilcoxon tests (α =0.05) 
	stat_gender_fix_count, p_gender_fix_count = wilcoxon(results_sex_mean.loc["M"]['Fixation Count'], results_sex_mean.loc["F"]['Fixation Count'])
	stat_gender_fix_time, p_gender_fix_time = wilcoxon(results_sex_mean.loc["M"]['Average Fixation Duration [ms]'], results_sex_mean.loc["F"]['Average Fixation Duration [ms]'])

	stat_age_fix_count, p_age_fix_count = mannwhitneyu(results_age_group_mean[results_age_group_mean['Age'] <= 22]['Fixation Count'], results_age_group_mean[results_age_group_mean['Age'] > 22]['Fixation Count'])
	stat_age_fix_time, p_age_fix_time = mannwhitneyu(results_age_group_mean[results_age_group_mean['Age'] <= 22]['Average Fixation Duration [ms]'], results_age_group_mean[results_age_group_mean['Age'] > 22]['Average Fixation Duration [ms]'])

	# We create a table to show the results
	data = [
		[
			'Avg. Fix. Duration (ms)', 
			f"{'%.2f' % results_sex_mean.loc['M']['Average Fixation Duration [ms]']}({'%.2f' % results_sex_std.loc['M']['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_sex_mean.loc['F']['Average Fixation Duration [ms]']}({'%.2f' % results_sex_std.loc['F']['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_age_mean.loc[Interval(18, 22)]['Average Fixation Duration [ms]']}({'%.2f' % results_age_std.loc[Interval(18, 22)]['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_age_mean.loc[Interval(22, 27)]['Average Fixation Duration [ms]']}({'%.2f' % results_age_std.loc[Interval(22, 27)]['Average Fixation Duration [ms]']})",
			'%.3f' % p_gender_fix_time,
			'%.3f' % p_age_fix_time
		],
		[
			'Fixation Count', 
			f"{'%.0f' % results_sex_mean.loc['M']['Fixation Count']} ({'%.0f' % results_sex_std.loc['M']['Fixation Count']})", 
			f"{'%.0f' % results_sex_mean.loc['F']['Fixation Count']} ({'%.0f' % results_sex_std.loc['F']['Fixation Count']})", 
			f"{'%.0f' % results_age_mean.loc[Interval(18, 22)]['Fixation Count']} ({'%.0f' % results_age_std.loc[Interval(18, 22)]['Fixation Count']})", 
			f"{'%.0f' % results_age_mean.loc[Interval(22, 27)]['Fixation Count']} ({'%.0f' % results_age_std.loc[Interval(22, 27)]['Fixation Count']})", 
			'%.3f' % p_gender_fix_count, 
			'%.3f' % p_age_fix_count
		]
	]

	data_df = pd.DataFrame(data)
	data_df.columns = ['', 'Men', 'Women', '18-22', '23-27', 'Gender p', 'Age p']
	return data_df

show_results2(results_pygaze)



                      Fixation Count  Average Fixation Duration [ms]   Age
CategoryName Subject                                                      
List         S151          38.200000                      178.571277  21.0
             S153          51.366667                      117.812847  19.0
             S154          66.433333                       93.470218  21.0
             S155          62.500000                      199.081273  20.0
             S156          40.466667                       81.234357  21.0
             S157          35.733333                       83.092020  22.0
             S159          24.033333                       68.770168  22.0
             S161          26.600000                       82.746223  18.0
             S162          75.366667                      128.599939  20.0
             S165          43.800000                      129.280585  21.0
             S167          50.466667                      126.595460  21.0
             S168        

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),106.46(62.30),131.38(91.41),112.87(41.39),123.72(62.95),1.000,0.625
1,Fixation Count,35 (29),40 (29),38 (21),38 (20),1.000,0.625


In [27]:
show_results2(results_ogama)

                      Fixation Count  Average Fixation Duration [ms]   Age
CategoryName Subject                                                      
List         S151          26.566667                      307.390180  21.0
             S153          35.833333                      193.524686  19.0
             S154          42.900000                      147.678644  21.0
             S155          53.733333                      240.638469  20.0
             S156          22.833333                      292.953152  21.0
             S157          24.233333                      258.006922  22.0
             S159          16.428571                      544.925076  22.0
             S161          22.758621                      306.961166  18.0
             S162          61.633333                      176.339091  20.0
             S165          37.533333                      174.800868  21.0
             S167          44.566667                      152.697115  21.0
             S168        

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),278.12(682.78),241.62(280.83),285.34(266.43),261.67(180.86),1.000,0.295
1,Fixation Count,28 (24),32 (23),29 (17),30 (17),1.000,0.619


PyGaze vs Ogama

In [39]:
# Function to make the file compatible with Ogama
import pandas as pd

# We load the original data from the file data/4/formatted-raw-data/151/ogama.txt
data = pd.read_csv('data/4/formatted-raw-data/153/ogama.txt', sep=',', skiprows=0, encoding='utf-16')

# Change the values of the ImageName column to remove the last 4 characters
data[' ImageName'] = data[' ImageName'].str[:-4]

# We remove all rows where Included? is "N"
data = data[data[' Included?'] == "Y"]

# Save it again as ogama2.csv
data.to_csv('data/4/formatted-raw-data/153/changed.csv', index=False)

Original Results of the Study

In [40]:
# Load the original data from the file results/original_data_4.csv
fixationsAll = pd.read_csv('results/original_data_4.csv')

# Look at only the data from the subject S153
fixationsAll = fixationsAll[fixationsAll['Subject'] == 'S153']

# Only keep the columns TrialID, FixationCount, AvgFixDuration and TotalFixationTime
fixationsAll = fixationsAll[['Image', 'FixationsCount', 'TotalFixationTime', 'AvgFixDuration']]
fixationsAll['FixationsCount'] = fixationsAll['FixationsCount'].astype('int32')
# Rename to Total Fixation Count, Total Fixation Duration and Average Fixation Duration
fixationsAll.columns = ['TrialID', 'Total Fixation Count', 'Total Fixation Duration', 'Average Fixation Duration']
#Sort by TrialID
fixationsAll = fixationsAll.sort_values(by=['TrialID'])
#fixationsAll = fixationsAll.style.hide()



Import data automatically into Ogama by manipulating the database 

In [1]:
import pandas as pd
import os
import sqlite3
import numpy as np
# Need to register numpy types to be able to save them in the database, as SQL only supports int8
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

# Path to database
database_path = 'G:\\Dokumente\\OgamaExperiments\\Experiment4_1\\Database\\Experiment4_1.db'

def import_data_into_ogama(path, subject):

	# We load the original data from the file data/4/formatted-raw-data/151/ogama.txt
	data = pd.read_csv(path, sep=',', skiprows=0, encoding='utf-16')

	# Change the values of the ImageName column to remove the last 4 characters
	data[' ImageName'] = data[' ImageName'].str[:-4]

	# We remove all rows where Included? is "N"
	data = data[data[' Included?'] == "Y"]

	# Normalize time
	data[' StartTime'] = data[' StartTime'] - data[' StartTime'].min()
	# Time as integer
	data[' StartTime'] = data[' StartTime'].astype('int32')

	# Format X and Y to screen of size 1920x1080 instead of 1024x768
	data[' X'] = data[' X'] * 1920 / 1024
	data[' Y'] = data[' Y'] * 1080 / 768

	# Connect to database
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	# Add a table for the subject called Subject + 'RawData'
	c.execute("CREATE TABLE IF NOT EXISTS ["+ subject +"Rawdata] ([ID] integer PRIMARY KEY AUTOINCREMENT NOT NULL,[SubjectName] varchar(50) NOT NULL COLLATE NOCASE, [TrialSequence] integer NOT NULL, [Time] integer NOT NULL, [PupilDiaX] float, [PupilDiaY] float, [GazePosX] float, [GazePosY] float, [MousePosX] float, [MousePosY] float, [EventID] integer)")

	# Add data in the following scheme:
		# SubjectName = subject
		# TrialSequence = ' ImageName'
		# Time = ' StartTime'
		# GazePosX = ' X'
		# GazePosY = ' Y'
	
	# For each row in the dataframe
	for index, row in data.iterrows():
		# Insert the data into the table
		c.execute("INSERT INTO ["+ subject +"Rawdata] (SubjectName, TrialSequence, Time, GazePosX, GazePosY) VALUES (?, ?, ?, ?, ?)", (subject, row[' ImageName'], row[' StartTime'], row[' X'], row[' Y']))

	# Add subject to the Subjects table
	c.execute("INSERT INTO Subjects (SubjectName) VALUES (?)", (subject,))

	# Add each grouped trial to the Trials table
	for trial in data.groupby(' ImageName'):
		trialstarttime = trial[1][' StartTime'].min()
		# Max Starttime + last duration
		trialduration = trial[1][' StartTime'].max() - trial[1][' StartTime'].min() + trial[1][' Duration'].iloc[-1]
		trialduration = int(trialduration)
		c.execute("INSERT INTO Trials (SubjectName, TrialID, TrialName, TrialSequence, Category,  TrialStartTime, Duration) VALUES (?, ?, ?, ?, ?, ?, ?)", (subject, trial[0], trial[0], trial[0], "",trialstarttime, trialduration))

	# Save the database
	conn.commit()

def drop_all_subject_tables():
	# Connect to database
	import sqlite3
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	for i in range(151, 205):
		# Drop the table named S + str(i) + "RawData"
		c.execute("DROP TABLE IF EXISTS S" + str(i) + "Rawdata")

	# Delete entries from Trials
	c.execute("DELETE FROM Trials")

	# Delete entries from Subjects
	c.execute("DELETE FROM Subjects")

	# Save the database
	conn.commit()

drop_all_subject_tables()

for i in range(151, 205):
	# If the folder i exists
	path = 'data/4/formatted-raw-data/' + str(i)
	if os.path.isdir(path):
		# Import the data into the database
		import_data_into_ogama(path + '/ogama.txt', 'S' + str(i))
		print('Imported data for subject S' + str(i))

Imported data for subject S151
Imported data for subject S152
Imported data for subject S153
Imported data for subject S154
Imported data for subject S155
Imported data for subject S156
Imported data for subject S157
Imported data for subject S159
Imported data for subject S160
Imported data for subject S161
Imported data for subject S162
Imported data for subject S163
Imported data for subject S165
Imported data for subject S167
Imported data for subject S168
Imported data for subject S169
Imported data for subject S171
Imported data for subject S172
Imported data for subject S173
Imported data for subject S174
Imported data for subject S175
Imported data for subject S176
Imported data for subject S177
Imported data for subject S179
Imported data for subject S182
Imported data for subject S183
Imported data for subject S200
Imported data for subject S201
Imported data for subject S202
Imported data for subject S203
Imported data for subject S204


Now enter Ogama, load the experiment that is using this database and run the fixation algorithm.

Import the fixations from the database and calculate our results

In [6]:
import pandas as pd
import os

# Path to database
database_path = 'G:\\Dokumente\\OgamaExperiments\\Experiment4_1\\Database\\Experiment4_1.db'

total_results = pd.DataFrame()

def calculate_results_for_subject(subject):
	# Connect to database
	import sqlite3
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	# Export all fixations for the subject
	c.execute("SELECT * FROM GazeFixations WHERE SubjectName = ?", (subject,))
	fixations = c.fetchall()

	# Save it into a dataframe
	fixations_df = pd.DataFrame(fixations)

	# Set the column names of the dataframe
	fixations_df.columns = ['ID', 'SubjectName', 'TrialID', 'TrialSequence', 'CountInTrial', 'StartTime', 'Length', 'PosX', 'PosY']

	# For each TrialID, we calculate the total fixation count, fixation duration and average fixation duration
	results = pd.DataFrame()
	results['Total Fixation Count'] = fixations_df.groupby('TrialID')['ID'].count()
	results['Total Fixation Duration'] = fixations_df.groupby('TrialID')['Length'].sum()
	results['Average Fixation Duration'] = fixations_df.groupby('TrialID')['Length'].mean()
	results['Subject'] = subject


	return results

	


for i in range(151, 205):
	# If the folder i exists
	path = 'data/4/formatted-raw-data/' + str(i)
	if os.path.isdir(path):
		# Append to the total_results dataframe
		total_results = total_results.append(calculate_results_for_subject('S' + str(i)))



# We load the original results from the file data/4/processed-data/fixationsAll.csv
fixationsAll = pd.read_csv('results/original_data_4.csv')

# We only keep the columns Subject, Sex, Age, Image, CategoryName, list_size, Likert_list_size, working_size, Likert_working_size, total_size, Likert_total_size and StimulusType
fixationsAll = fixationsAll[['Subject', 'Sex', 'Age', 'Image', 'CategoryName', 'list_size', 'Likert_list_size', 'working_size', 'Likert_working_size', 'total_size', 'Likert_total_size', 
'StimulusType']]
fixationsAll['Subject'] = fixationsAll['Subject'].astype("string")
fixationsAll['Image'] = fixationsAll['Image'].astype('int32')


# We combine the two dataframes into one by combining the columns where the values of Participant and Task are the same
results_ogama = pd.merge(fixationsAll, total_results, how='inner', left_on=['Subject', 'Image'], right_on=['Subject', 'TrialID'])
# Rename the columns Fixation Count, Total Fixation Duration and Average Fixation Duration
results_ogama = results_ogama.rename(columns={'Total Fixation Count': 'Fixation Count', 'Total Fixation Duration': 'Total Fixation Duration [ms]', 'Average Fixation Duration': 'Average Fixation Duration [ms]'})

results_ogama


C:\Users\Timon\AppData\Local\Temp\ipykernel_20236\2686446938.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_results = total_results.append(calculate_results_for_subject('S' + str(i)))
C:\Users\Timon\AppData\Local\Temp\ipykernel_20236\2686446938.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_results = total_results.append(calculate_results_for_subject('S' + str(i)))
C:\Users\Timon\AppData\Local\Temp\ipykernel_20236\2686446938.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_results = total_results.append(calculate_results_for_subject('S' + str(i)))
C:\Users\Timon\AppData\Local\Temp\ipykernel_20236\2686446938.py:43: FutureWarning: The frame.append method is deprecated and will be removed from

,Subject,Sex,Age,Image,CategoryName,list_size,Likert_list_size,working_size,Likert_working_size,total_size,Likert_total_size,StimulusType,Fixation Count,Total Fixation Duration [ms],Average Fixation Duration [ms]
0,S151,M,21.0,8,List,6.5,1,16.0,4.0,12.0,1.0,ainsert,13,2335,179.615385
1,S151,M,21.0,25,List,13.0,3,6.0,1.0,13.0,1.0,ainsert,15,4088,272.533333
2,S151,M,21.0,28,List,14.0,3,8.0,2.0,14.0,2.0,ainsert,39,5059,129.717949
3,S151,M,21.0,21,List,14.0,3,10.0,2.0,14.0,2.0,ainsert,21,5893,280.619048
4,S151,M,21.0,11,List,17.0,4,13.0,3.0,17.0,3.0,ainsert,23,5239,227.782609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2521,S204,M,24.0,43,Tree,9.0,2,9.0,3.0,1.0,2.0,traversal,24,3839,159.958333
2522,S204,M,24.0,49,Tree,9.0,2,9.0,3.0,1.0,2.0,traversal,32,5124,160.125000
2523,S204,M,24.0,40,Tree,12.0,2,12.0,3.0,12.0,2.0,traversal,53,7424,140.075472
2524,S204,M,24.0,55,Tree,13.0,2,13.0,3.0,13.0,2.0,traversal,30,6360,212.000000


In [60]:
data_df = show_results1(results_pygaze)
data_df

CategoryName  Subject
List          S151       157152.0
              S153       183849.4
              S154       188985.1
              S155       373423.8
              S156       100368.1
                           ...   
Tree          S200        77974.4
              S201       230751.5
              S202        60479.5
              S203        97952.2
              S204       165780.5
Name: Total Fixation Duration [ms], Length: 92, dtype: float64
Subject
S151    157152.0
S153    183849.4
S154    188985.1
S155    373423.8
S156    100368.1
S157     93552.9
S159     62329.7
S160    299862.8
S161     77742.8
S162    298542.0
S163    234851.4
S165    176929.3
S167    196654.5
S168    102585.8
S169    187850.4
S171      7241.5
S172     93115.5
S173    308305.5
S174    237502.8
S175    216507.6
S176    197226.9
S177    252634.3
S179    387926.8
S182     73396.2
S183    216424.4
S200     84835.5
S201    246592.3
S202     61411.5
S203     95082.3
S204    230525.4
Name: Total Fixation Du

c:\Users\Timon\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:3337: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,,List,Mental,Tree,List vs Mental p,Tree vs Mental p
0,Fixation Time (s),181.45(96.70),71.70(47.94),170.85(109.38),0.000,0.000
1,Fixation Count,49 (27),21 (14),47 (24),0.000,0.000


In [7]:
data_df = show_results1(results_ogama)
data_df

,,List,Mental,Tree,List vs Mental p,Tree vs Mental p
0,Fixation Time (s),217.87(90.37),83.98(52.45),203.77(104.47),0.000,0.000
1,Fixation Count,38 (21),17 (11),36 (19),0.000,0.000


![Results Study 4 1](images/results_study_4_1.png)

In [5]:
data_df = show_results2(results_pygaze)
data_df

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),106.46(62.30),131.38(91.41),113.91(65.61),123.76(91.83),0.000,0.002
1,Fixation Count,35 (29),40 (29),38 (29),38 (30),0.000,0.249


In [7]:
data_df = show_results2(results_ogama)
data_df

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),278.12(682.78),241.62(280.83),262.84(555.82),263.60(584.99),0.014,0.009
1,Fixation Count,28 (24),32 (23),29 (23),31 (25),0.000,0.079


![Results Study 4 2](images/results_study_4_2.png)